# Set Up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/egh_vlm.zip

Archive:  /content/egh_vlm.zip
   creating: egh_vlm/
  inflating: egh_vlm/extract_feature.py  
  inflating: egh_vlm/generate_answers.py  
  inflating: egh_vlm/hallucination_dataset.py  
  inflating: egh_vlm/hallucination_detector.py  
  inflating: egh_vlm/training.py     
  inflating: egh_vlm/utils.py        


In [1]:
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

from egh_vlm.hallucination_detector import DetectorModule
from egh_vlm.hallucination_dataset import stratified_split, hallucination_collate_fn
from egh_vlm.training import get_features, eval_detector, train

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--dataset_dir_path", type=str, default="/content/drive/MyDrive/Bachelor's Thesis/DH-7 2026-TTNT-HallucinationDetection/Dataset/HallusionBench")
parser.add_argument("--features_file_path", type=str, default="/content/drive/MyDrive/Bachelor's Thesis/DH-7 2026-TTNT-HallucinationDetection/Dataset/HallusionBench/features.pt")
parser.add_argument("--detector_file_path", type=str, default="/content/drive/MyDrive/Bachelor's Thesis/DH-7 2026-TTNT-HallucinationDetection/Dataset/HallusionBench/detector.pt")
parser.add_argument("--model_name", type=str, default="Qwen/Qwen3-VL-2B-Instruct")
parser.add_argument("--train_ratio", type=float, default=0.7)
args, _ = parser.parse_known_args()

# Training

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Qwen3VLForConditionalGeneration.from_pretrained(
    args.model_name,
    dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(
    args.model_name,
    min_pixels=256 * 28 * 28,
    max_pixels=512 * 28 * 28)

In [ ]:
dataset, hidden_size = get_features(args.features_file_path, args.dataset_dir_path, model, processor, device)
train_dataset, test_dataset = stratified_split(dataset, args.train_ratio, random_state=42)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=16,
    collate_fn=hallucination_collate_fn,
    shuffle=True,
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=16,
    collate_fn=hallucination_collate_fn,
    shuffle=True,
)

Successfully load the Hallusion Bench dataset with: 500 samples.


Extract features::  43%|████▎     | 213/500 [47:48<1:13:13, 15.31s/it]

In [ ]:
detector = DetectorModule(hidden_size, hidden_size, 1, 0.2)
epoch = 20
loss_function = nn.BCELoss()
optim = torch.optim.Adam(detector.parameters(), lr=1e-4)
best_acc = 0.0
best_f1 = 0.0

for i in range(epoch):
    total_loss = train(detector, loss_function, optim, train_dataloader)
    print(f'Epoch [{i + 1}/{epoch}], Loss: {total_loss / 2000:.4f}')
    acc, f1, pr_auc = eval_detector(detector, test_dataloader)
    print(f'Epoch [{i + 1}/{epoch}], ACC: {acc:.4f}, F1: {f1:.4f}, PR-AUC:{pr_auc:.4f}')

    if acc > best_acc:
        best_acc = acc
        print(f'Best ACC at epoch {epoch + 1}')
    if f1 > best_f1:
        best_f1 = f1
        print(f'Best F1 at epoch {epoch + 1}')
    if total_loss < 1e-4:
        break

print(f"Eval accuracy: {best_acc:.4f}, F1: {best_f1:.4f}")
print(f'Finished!')

In [ ]:
torch.save(detector.state_dict(), args.detector_file_path)